## Final Project  

Bioinformatics & Network Medicine  

Group 14  

Emre Pelzer                  pelzer.2183479@studenti.uniroma1.it  
Syed Habibul Bashar          bashar.2102742@studenti.uniroma1.it  

Libraries

In [5]:
import os
import random
import itertools
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from scipy.sparse import csgraph
from scipy.stats import hypergeom
from sklearn.model_selection import KFold
from scipy.sparse.linalg import expm_multiply
from sklearn.metrics import precision_score, recall_score, f1_score

### 1. Intractome Construction

In [57]:

file_path = r'E:\University\BINM\BIOGRID-ALL-4.4.240.tab3\BIOGRID-ALL-4.4.240.tab3.txt'

df = pd.read_csv(file_path, delimiter='\t')

df.head()


C:\Users\Syed Habibul Bashar\AppData\Local\Temp\ipykernel_15192\3348542099.py:3: DtypeWarning: Columns (1,2,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, delimiter='\t')


,#BioGRID Interaction ID,Entrez Gene Interactor A,Entrez Gene Interactor B,BioGRID ID Interactor A,BioGRID ID Interactor B,Systematic Name Interactor A,Systematic Name Interactor B,Official Symbol Interactor A,Official Symbol Interactor B,Synonyms Interactor A,Synonyms Interactor B,Experimental System,Experimental System Type,Author,Publication Source,Organism ID Interactor A,Organism ID Interactor B,Throughput,Score,Modification,Qualifications,Tags,Source Database,SWISS-PROT Accessions Interactor A,TREMBL Accessions Interactor A,REFSEQ Accessions Interactor A,SWISS-PROT Accessions Interactor B,TREMBL Accessions Interactor B,REFSEQ Accessions Interactor B,Ontology Term IDs,Ontology Term Names,Ontology Term Categories,Ontology Term Qualifier IDs,Ontology Term Qualifier Names,Ontology Term Types,Organism Name Interactor A,Organism Name Interactor B
0,103,6416,2318,112315,108607,-,-,MAP2K4,FLNC,JNKK|JNKK1|MAPKK4|MEK4|MKK4|PRKMK4|SAPKK-1|SAP...,ABP-280|ABP280A|ABPA|ABPL|FLN2|MFM5|MPD4,Two-hybrid,physical,Marti A (1997),PUBMED:9006895,9606,9606,Low Throughput,-,-,-,-,BIOGRID,P45985,-,NP_003001|NP_001268364,Q14315,Q59H94,NP_001120959|NP_001449,-,-,-,-,-,-,Homo sapiens,Homo sapiens
1,117,84665,88,124185,106603,-,-,MYPN,ACTN2,CMD1DD|CMH22|MYOP|RCM4,CMD1AA,Two-hybrid,physical,Bang ML (2001),PUBMED:11309420,9606,9606,Low Throughput,-,-,-,-,BIOGRID,Q86TC9,A0A087WX60,NP_001243197|NP_001243196|NP_115967,P35609,Q59FD9|F6THM6,NP_001094|NP_001265272|NP_001265273,-,-,-,-,-,-,Homo sapiens,Homo sapiens
2,183,90,2339,106605,108625,-,-,ACVR1,FNTA,ACTRI|ACVR1A|ACVRLK2|ALK2|FOP|SKR1|TSRI,FPTA|PGGT1A|PTAR2,Two-hybrid,physical,Wang T (1996),PUBMED:8599089,9606,9606,Low Throughput,-,-,-,-,BIOGRID,Q04771,D3DPA4,NP_001104537|NP_001096,P49354,-,NP_002018,-,-,-,-,-,-,Homo sapiens,Homo sapiens
3,278,2624,5371,108894,111384,-,-,GATA2,PML,DCML|IMD21|MONOMAC|NFE1B,MYL|PP8675|RNF71|TRIM19,Two-hybrid,physical,Tsuzuki S (2000),PUBMED:10938104,9606,9606,Low Throughput,-,-,-,-,BIOGRID,P23769,-,NP_001139134|NP_116027|NP_001139133,P29590,-,NP_150250|NP_150253|NP_150252|NP_150247|NP_150...,-,-,-,-,-,-,Homo sapiens,Homo sapiens
4,418,6118,6774,112038,112651,RP4-547C9.3,-,RPA2,STAT3,REPA2|RP-A p32|RP-A p34|RPA32,ADMIO|APRF|HIES,Two-hybrid,physical,Kim J (2000),PUBMED:10875894,9606,9606,Low Throughput,-,-,-,-,BIOGRID,P15927,B4DUL2,NP_001342057|NP_002937|NP_001284487|NP_0013420...,P40763,-,NP_644805|NP_003141|NP_001356447|NP_001356443|...,-,-,-,-,-,-,Homo sapiens,Homo sapiens


In [58]:
# Filter for human-human interactions (taxonomy ID for humans is 9606) 
human_interactions = df[(df['Organism ID Interactor A'] == 9606) & (df['Organism ID Interactor B'] == 9606)] 
# Print the first 5 rows of the filtered DataFrame
human_interactions.head()

,#BioGRID Interaction ID,Entrez Gene Interactor A,Entrez Gene Interactor B,BioGRID ID Interactor A,BioGRID ID Interactor B,Systematic Name Interactor A,Systematic Name Interactor B,Official Symbol Interactor A,Official Symbol Interactor B,Synonyms Interactor A,Synonyms Interactor B,Experimental System,Experimental System Type,Author,Publication Source,Organism ID Interactor A,Organism ID Interactor B,Throughput,Score,Modification,Qualifications,Tags,Source Database,SWISS-PROT Accessions Interactor A,TREMBL Accessions Interactor A,REFSEQ Accessions Interactor A,SWISS-PROT Accessions Interactor B,TREMBL Accessions Interactor B,REFSEQ Accessions Interactor B,Ontology Term IDs,Ontology Term Names,Ontology Term Categories,Ontology Term Qualifier IDs,Ontology Term Qualifier Names,Ontology Term Types,Organism Name Interactor A,Organism Name Interactor B
0,103,6416,2318,112315,108607,-,-,MAP2K4,FLNC,JNKK|JNKK1|MAPKK4|MEK4|MKK4|PRKMK4|SAPKK-1|SAP...,ABP-280|ABP280A|ABPA|ABPL|FLN2|MFM5|MPD4,Two-hybrid,physical,Marti A (1997),PUBMED:9006895,9606,9606,Low Throughput,-,-,-,-,BIOGRID,P45985,-,NP_003001|NP_001268364,Q14315,Q59H94,NP_001120959|NP_001449,-,-,-,-,-,-,Homo sapiens,Homo sapiens
1,117,84665,88,124185,106603,-,-,MYPN,ACTN2,CMD1DD|CMH22|MYOP|RCM4,CMD1AA,Two-hybrid,physical,Bang ML (2001),PUBMED:11309420,9606,9606,Low Throughput,-,-,-,-,BIOGRID,Q86TC9,A0A087WX60,NP_001243197|NP_001243196|NP_115967,P35609,Q59FD9|F6THM6,NP_001094|NP_001265272|NP_001265273,-,-,-,-,-,-,Homo sapiens,Homo sapiens
2,183,90,2339,106605,108625,-,-,ACVR1,FNTA,ACTRI|ACVR1A|ACVRLK2|ALK2|FOP|SKR1|TSRI,FPTA|PGGT1A|PTAR2,Two-hybrid,physical,Wang T (1996),PUBMED:8599089,9606,9606,Low Throughput,-,-,-,-,BIOGRID,Q04771,D3DPA4,NP_001104537|NP_001096,P49354,-,NP_002018,-,-,-,-,-,-,Homo sapiens,Homo sapiens
3,278,2624,5371,108894,111384,-,-,GATA2,PML,DCML|IMD21|MONOMAC|NFE1B,MYL|PP8675|RNF71|TRIM19,Two-hybrid,physical,Tsuzuki S (2000),PUBMED:10938104,9606,9606,Low Throughput,-,-,-,-,BIOGRID,P23769,-,NP_001139134|NP_116027|NP_001139133,P29590,-,NP_150250|NP_150253|NP_150252|NP_150247|NP_150...,-,-,-,-,-,-,Homo sapiens,Homo sapiens
4,418,6118,6774,112038,112651,RP4-547C9.3,-,RPA2,STAT3,REPA2|RP-A p32|RP-A p34|RPA32,ADMIO|APRF|HIES,Two-hybrid,physical,Kim J (2000),PUBMED:10875894,9606,9606,Low Throughput,-,-,-,-,BIOGRID,P15927,B4DUL2,NP_001342057|NP_002937|NP_001284487|NP_0013420...,P40763,-,NP_644805|NP_003141|NP_001356447|NP_001356443|...,-,-,-,-,-,-,Homo sapiens,Homo sapiens


In [59]:
# Filter for human-human interactions
human_physical_interactions = human_interactions[(human_interactions['Experimental System Type'] == 'physical')] 
# Print the first 5 rows of the filtered DataFrame
human_physical_interactions.head()

,#BioGRID Interaction ID,Entrez Gene Interactor A,Entrez Gene Interactor B,BioGRID ID Interactor A,BioGRID ID Interactor B,Systematic Name Interactor A,Systematic Name Interactor B,Official Symbol Interactor A,Official Symbol Interactor B,Synonyms Interactor A,Synonyms Interactor B,Experimental System,Experimental System Type,Author,Publication Source,Organism ID Interactor A,Organism ID Interactor B,Throughput,Score,Modification,Qualifications,Tags,Source Database,SWISS-PROT Accessions Interactor A,TREMBL Accessions Interactor A,REFSEQ Accessions Interactor A,SWISS-PROT Accessions Interactor B,TREMBL Accessions Interactor B,REFSEQ Accessions Interactor B,Ontology Term IDs,Ontology Term Names,Ontology Term Categories,Ontology Term Qualifier IDs,Ontology Term Qualifier Names,Ontology Term Types,Organism Name Interactor A,Organism Name Interactor B
0,103,6416,2318,112315,108607,-,-,MAP2K4,FLNC,JNKK|JNKK1|MAPKK4|MEK4|MKK4|PRKMK4|SAPKK-1|SAP...,ABP-280|ABP280A|ABPA|ABPL|FLN2|MFM5|MPD4,Two-hybrid,physical,Marti A (1997),PUBMED:9006895,9606,9606,Low Throughput,-,-,-,-,BIOGRID,P45985,-,NP_003001|NP_001268364,Q14315,Q59H94,NP_001120959|NP_001449,-,-,-,-,-,-,Homo sapiens,Homo sapiens
1,117,84665,88,124185,106603,-,-,MYPN,ACTN2,CMD1DD|CMH22|MYOP|RCM4,CMD1AA,Two-hybrid,physical,Bang ML (2001),PUBMED:11309420,9606,9606,Low Throughput,-,-,-,-,BIOGRID,Q86TC9,A0A087WX60,NP_001243197|NP_001243196|NP_115967,P35609,Q59FD9|F6THM6,NP_001094|NP_001265272|NP_001265273,-,-,-,-,-,-,Homo sapiens,Homo sapiens
2,183,90,2339,106605,108625,-,-,ACVR1,FNTA,ACTRI|ACVR1A|ACVRLK2|ALK2|FOP|SKR1|TSRI,FPTA|PGGT1A|PTAR2,Two-hybrid,physical,Wang T (1996),PUBMED:8599089,9606,9606,Low Throughput,-,-,-,-,BIOGRID,Q04771,D3DPA4,NP_001104537|NP_001096,P49354,-,NP_002018,-,-,-,-,-,-,Homo sapiens,Homo sapiens
3,278,2624,5371,108894,111384,-,-,GATA2,PML,DCML|IMD21|MONOMAC|NFE1B,MYL|PP8675|RNF71|TRIM19,Two-hybrid,physical,Tsuzuki S (2000),PUBMED:10938104,9606,9606,Low Throughput,-,-,-,-,BIOGRID,P23769,-,NP_001139134|NP_116027|NP_001139133,P29590,-,NP_150250|NP_150253|NP_150252|NP_150247|NP_150...,-,-,-,-,-,-,Homo sapiens,Homo sapiens
4,418,6118,6774,112038,112651,RP4-547C9.3,-,RPA2,STAT3,REPA2|RP-A p32|RP-A p34|RPA32,ADMIO|APRF|HIES,Two-hybrid,physical,Kim J (2000),PUBMED:10875894,9606,9606,Low Throughput,-,-,-,-,BIOGRID,P15927,B4DUL2,NP_001342057|NP_002937|NP_001284487|NP_0013420...,P40763,-,NP_644805|NP_003141|NP_001356447|NP_001356443|...,-,-,-,-,-,-,Homo sapiens,Homo sapiens


In [49]:
# Save the filtered DataFrame to a new CSV file 
output_path = r'E:\University\BINM\BIOGRID-Human-Human-Physical-Interactions.csv' 
human_physical_interactions.to_csv(output_path)
print("Filtered DataFrame has been saved successfully!")

Filtered DataFrame has been saved successfully!


removing self loops and redundents

In [60]:
# Remove self-loops
human_physical_interactions = human_physical_interactions[
    human_physical_interactions['Entrez Gene Interactor A'] != human_physical_interactions['Entrez Gene Interactor B']
]

# Convert the columns to the same type (string or integer)
human_physical_interactions['Entrez Gene Interactor A'] = human_physical_interactions['Entrez Gene Interactor A'].astype(str)
human_physical_interactions['Entrez Gene Interactor B'] = human_physical_interactions['Entrez Gene Interactor B'].astype(str)

# Sort interactors to handle duplicates regardless of order
# Create a new DataFrame with sorted interactors for consistent comparison
sorted_interactions = human_physical_interactions.apply(
    lambda x: pd.Series(sorted([x['Entrez Gene Interactor A'], x['Entrez Gene Interactor B']])), axis=1
)

# Rename the new columns for clarity
sorted_interactions.columns = ['Interactor A', 'Interactor B']

# Combine the sorted interactions with the original DataFrame to ensure we keep all columns
human_physical_interactions['Interactor A'] = sorted_interactions['Interactor A']
human_physical_interactions['Interactor B'] = sorted_interactions['Interactor B']

# Drop duplicates based on the sorted interactors
human_physical_interactions = human_physical_interactions.drop_duplicates(subset=['Interactor A', 'Interactor B'])

#drop the added 'Interactor A' and 'Interactor B' columns if they are no longer needed
human_physical_interactions = human_physical_interactions.drop(columns=['Interactor A', 'Interactor B'])

# Save the cleaned DataFrame to a new file
output_path_cleaned = r'E:\University\BINM\BIOGRID-Human-Human-Physical-Interactions-Cleaned.csv'
human_physical_interactions.to_csv(output_path_cleaned, index=False)

print("Redundant interactions and self-loops have been removed successfully!")


Redundant interactions and self-loops have been removed successfully!


In [78]:
file_path = r'E:\University\BINM\BIOGRID-Human-Human-Physical-Interactions-Cleaned.csv'

interactome = pd.read_csv(file_path)

C:\Users\Syed Habibul Bashar\AppData\Local\Temp\ipykernel_15192\491096485.py:3: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactome = pd.read_csv(file_path)


LCC (largest connected componect)

In [80]:
# Create a graph from the interactome
G = nx.Graph()
for _, row in interactome.iterrows():
    # Add edges between interacting proteins
    G.add_edge(row['Official Symbol Interactor A'], row['Official Symbol Interactor B'])

# Find the Largest Connected Component (LCC)
largest_cc = max(nx.connected_components(G), key=len)
interactome_lcc = G.subgraph(largest_cc)

# Output basic statistics about the LCC
print(f"Number of nodes in LCC: {interactome_lcc.number_of_nodes()}")
print(f"Number of edges in LCC: {interactome_lcc.number_of_edges()}")

Number of nodes in LCC: 19972
Number of edges in LCC: 862217


In [165]:
interactome_nodes = pd.DataFrame(interactome_lcc.nodes)
# Save the DataFrame to a TSV file 
outputpath = r'E:\University\BINM\all_interactome_genes.tsv'
interactome_nodes.to_csv(outputpath, sep='\t', index=False) 
print("Disease genes saved to 'all_interactome_genes.tsv'")

Disease genes saved to 'all_interactome_genes.tsv'


GDAs

In [83]:
file_path = r'E:\University\BINM\Osteoporoses_DISEASES_Summary_GDA_CURATED_C0029456 - DISEASES_Summary_GDA_CURATED_C0029456.csv'

dfo = pd.read_csv(file_path)

dfo.head()

,DOClass,Disease,DiseaseClass,EvidenceIndexGDA,EvidenceLevelGDA,FirstRef,Gene,GeneDPI,GeneDSI,GeneFullName,GenePLI,HPOClass,LastRef,NumDiseasesAssociatedToGene,NumGenesDisease,NumPMIDs,NumVariantsAssociatedToGene,NumVariantsDisease,NumVariantsGDA,PathwayIDs,PathwayNames,ProteinClass,ScoreGDA,SemanticType,Type,UnitProt,diseaseUMLSCUI,geneEnsemblIDs,geneNcbiID,geneNcbiType,numCTs,numberChemsIncludedInEvidence,numberPmidsWithChemsIncludedInEvidence
0,disease of anatomical entity (7),Osteoporoses,"Nutritional and Metabolic Diseases,Musculoskel...",9.437.751.004.016.060,NaN,2005.0,PTH,1.31,0.35,parathyroid hormone,784.32.00,Abnormality of the musculoskeletal system (33127),2019.0,27,87,6,10,225,0,R-HSA-162582,Signal Transduction,NaN,1.0,Disease or Syndrome (T047),disease,P01270,C0029456,ENSG00000152266,5741,protein-coding,80,2,2
1,disease of anatomical entity (7),Osteoporoses,"Nutritional and Metabolic Diseases,Musculoskel...",9.649.122.807.017.540,NaN,2004.0,LRP5,1.31,0.46,LDL receptor related protein 5,1561.03.00,Abnormality of the musculoskeletal system (33127),2005.0,41,87,3,330,225,159,"R-HSA-162582,R-HSA-1643685","Signal Transduction,Disease",NaN,1.0,Disease or Syndrome (T047),disease,O75197,C0029456,ENSG00000162337,4041,protein-coding,0,0,0
2,disease of anatomical entity (7),Osteoporoses,"Nutritional and Metabolic Diseases,Musculoskel...",1.0,NaN,2013.0,WNT1,1.36,0.30,Wnt family member 1,452.27.00,Abnormality of the musculoskeletal system (33127),2013.0,8,87,2,28,225,2,"R-HSA-1266738,R-HSA-162582","Developmental Biology,Signal Transduction",Signaling,1.0,Disease or Syndrome (T047),disease,P04628,C0029456,ENSG00000125084,7471,protein-coding,1,0,0
3,disease of anatomical entity (7),Osteoporoses,"Nutritional and Metabolic Diseases,Musculoskel...",9.464.285.714.285.710,NaN,2007.0,TNFSF11,1.31,0.40,TNF superfamily member 11,1097.47.00,Abnormality of the musculoskeletal system (33127),2007.0,11,87,1,50,225,0,R-HSA-168256,Immune System,Signaling,1.0,Disease or Syndrome (T047),disease,"Q5T9Y4,O14788,Q54A98",C0029456,ENSG00000120659,8600,protein-coding,18,0,0
4,disease of anatomical entity (7),Osteoporoses,"Nutritional and Metabolic Diseases,Musculoskel...",9.119.496.855.345.910,NaN,2007.0,TNFRSF11B,1.31,0.38,TNF receptor superfamily member 11b,1469.04.00,Abnormality of the musculoskeletal system (33127),2007.0,13,87,1,49,225,0,R-HSA-168256,Immune System,NaN,1.0,Disease or Syndrome (T047),disease,O00300,C0029456,ENSG00000164761,4982,protein-coding,21,0,0


In [4]:

file_path = r'E:\University\BINM\hgnc_custom_set_2024-12-03.txt'

dfh = pd.read_csv(file_path, delimiter='\t')

dfh.head()

,HGNC ID,Approved symbol,Approved name,Status,Previous symbols,Alias symbols,Chromosome,Accession numbers,RefSeq IDs,UniProt ID(supplied by UniProt),NCBI Gene ID,Ensembl gene ID
0,HGNC:5,A1BG,alpha-1-B glycoprotein,Approved,NaN,NaN,19q13.43,NaN,NM_130786,P04217,1.0,ENSG00000121410
1,HGNC:37133,A1BG-AS1,A1BG antisense RNA 1,Approved,"NCRNA00181, A1BGAS, A1BG-AS",FLJ23569,19q13.43,BC040926,NR_015380,NaN,503538.0,ENSG00000268895
2,HGNC:24086,A1CF,APOBEC1 complementation factor,Approved,NaN,"ACF, ASP, ACF64, ACF65, APOBEC1CF",10q11.23,AF271790,NM_014576,Q9NQ94,29974.0,ENSG00000148584
3,HGNC:6,A1S9T,"symbol withdrawn, see [HGNC:12469](/data/gene-...",Symbol Withdrawn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,HGNC:7,A2M,alpha-2-macroglobulin,Approved,NaN,"FWP007, S863-7, CPAMD5",12p13.31,"BX647329, X68728, M11313",NM_000014,P01023,2.0,ENSG00000175899


In [23]:
# Merge the data frames on the gene columns 
merged_df = dfo.merge(dfh, left_on='Gene', right_on='Approved symbol', how='left', suffixes=('_dfo', '_dfh'))
merged_df.count()

DOClass                                   87
Disease                                   87
DiseaseClass                              87
EvidenceIndexGDA                          81
EvidenceLevelGDA                           0
FirstRef                                  59
Gene                                      87
GeneDPI                                   87
GeneDSI                                   87
GeneFullName                              87
GenePLI                                   80
HPOClass                                  87
LastRef                                   59
NumDiseasesAssociatedToGene               87
NumGenesDisease                           87
NumPMIDs                                  87
NumVariantsAssociatedToGene               87
NumVariantsDisease                        87
NumVariantsGDA                            87
PathwayIDs                                78
PathwayNames                              78
ProteinClass                              50
ScoreGDA  

In [56]:
# Convert relevant columns to the same data type (string)
dfo['geneEnsemblIDs'] = dfo['geneEnsemblIDs'].astype(str)
dfo['geneNcbiID'] = dfo['geneNcbiID'].astype(str)  # Format to 1 decimal place
dfo['UnitProt'] = dfo['UnitProt'].astype(str)

dfh['Ensembl gene ID'] = dfh['Ensembl gene ID'].astype(str)
dfh['NCBI Gene ID'] = dfh['NCBI Gene ID'].astype(str) # Format to 1 decimal place
dfh['UniProt ID(supplied by UniProt)'] = dfh['UniProt ID(supplied by UniProt)'].astype(str)

# Merge the data frames on the gene columns
merged_df = dfo.merge(dfh, left_on='Gene', right_on='Approved symbol', how='left', suffixes=('_dfo', '_dfh'))

# Function to check if there's at least one matching UniProt ID
def check_unitprot_match(row):
    dfo_ids = set(row['UnitProt'].split(','))
    dfh_id = row['UniProt ID(supplied by UniProt)']
    return dfh_id in dfo_ids

# Check for inconsistencies in the specified columns
inconsistencies = merged_df[(merged_df['geneEnsemblIDs'] != merged_df['Ensembl gene ID']) |
                            (merged_df['geneNcbiID'] != merged_df['NCBI Gene ID']) |
                            (~merged_df.apply(check_unitprot_match, axis=1))]

# Print the inconsistencies
print("Inconsistencies found:")
print(inconsistencies[['Gene', 'geneEnsemblIDs', 'Ensembl gene ID', 
                       'geneNcbiID', 'NCBI Gene ID', 'UnitProt', 'UniProt ID(supplied by UniProt)']])


Inconsistencies found:
        Gene      geneEnsemblIDs  Ensembl gene ID   geneNcbiID  NCBI Gene ID  \
12         CALCA  ENSG00000110680  ENSG00000110680        796.0     796.0      
84  LOC130059892                               NaN  130059892.0       NaN      
86  LOC126862586                               NaN  126862586.0       NaN      

      UnitProt    UniProt ID(supplied by UniProt)  
12  P01258,P06881          P01258, P06881          
84                                    NaN          
86                                    NaN          


1.3 finding genes in LCC

In [ ]:
# Load disease-gene associations
disease_genes = dfo
disease_gene_set = set(disease_genes['Gene'])  # Extract disease genes into a set for quick lookup

# Extract nodes from the LCC
lcc_nodes = set(interactome_lcc.nodes)

# Find disease genes present in the LCC
disease_genes_in_lcc = lcc_nodes.intersection(disease_gene_set)

# Output results
print(f"Number of disease genes in the LCC: {len(disease_genes_in_lcc)}")
print("Disease genes in the LCC:")
print(disease_genes_in_lcc)

#Save the list of disease genes in the LCC
pd.DataFrame({'Disease Genes in LCC': list(disease_genes_in_lcc)}).to_csv('disease_genes_in_lcc.csv', index=False)


Number of disease genes in the LCC: 84
Disease genes in the LCC:
{'REN', 'CCT2', 'LEPR', 'GSN', 'OXCT1', 'KL', 'PKM', 'ANXA2', 'SOD2', 'TNFSF11', 'TNFRSF1B', 'BGLAP', 'LTF', 'TNFRSF11A', 'ANTXR2', 'P4HB', 'GPC6', 'CALCR', 'TUBA1C', 'IGF1R', 'CYP17A1', 'RYR1', 'CA2', 'GAPDH', 'MGLL', 'BMP4', 'ADCY5', 'CYP24A1', 'CAP1', 'TLN1', 'PLEK', 'PDLIM4', 'OPA1', 'IL6R', 'IL32', 'FGA', 'COL1A1', 'RAB7B', 'FGB', 'BMP2', 'RSU1', 'TGFB1', 'PGLS', 'PNP', 'DBP', 'GPX1', 'TPI1', 'COL1A2', 'PSMA5', 'LRP5', 'PSMA2', 'DAAM2', 'ESR2', 'ENO1', 'VCL', 'GSTP1', 'CXCR4', 'TUBA1B', 'PRDX3', 'IL6', 'WNT1', 'CYP19A1', 'GPD2', 'TNFRSF11B', 'ACTG1', 'TPM4', 'SERPINF1', 'PARK7', 'GORAB', 'ZDHHC13', 'IL1RN', 'POMC', 'ATIC', 'NRIP1', 'ESR1', 'WDR1', 'CALCA', 'SRC', 'LEP', 'PTH', 'IDH2', 'U2AF1', 'IL1A', 'CLEC11A'}


In [218]:
# Save the list of disease genes in the LCC as a TSV
pd.DataFrame({'Disease Genes in LCC': list(disease_genes_in_lcc)}).to_csv('disease_genes_in_lcc.tsv', sep="\t", index=False)
print("Disease genes in the LCC have been saved as a TSV file: 'disease_genes_in_lcc.tsv'")


Disease genes in the LCC have been saved as a TSV file: 'disease_genes_in_lcc.tsv'


Desease LCC

In [ ]:
# Filter edges in the LCC for disease genes
disease_interactions = [
    (u, v) for u, v in interactome_lcc.edges
    if u in disease_genes_in_lcc and v in disease_genes_in_lcc
]

# Create a graph for the disease interactome
G_disease = nx.Graph()
G_disease.add_edges_from(disease_interactions)

# Find the Largest Connected Component (LCC) of the disease interactome
largest_cc_disease = max(nx.connected_components(G_disease), key=len)
disease_lcc = G_disease.subgraph(largest_cc_disease)

# Output basic statistics
print(f"Number of nodes in the Disease LCC: {disease_lcc.number_of_nodes()}")
print(f"Number of edges in the Disease LCC: {disease_lcc.number_of_edges()}")


Number of nodes in the Disease LCC: 47
Number of edges in the Disease LCC: 113


In [87]:
# Compute node degree
node_degree = dict(disease_lcc.degree())

# Compute betweenness centrality
betweenness_centrality = nx.betweenness_centrality(disease_lcc)

# Compute eigenvector centrality
eigenvector_centrality = nx.eigenvector_centrality(disease_lcc)

# Compute closeness centrality
closeness_centrality = nx.closeness_centrality(disease_lcc)

# Avoid dividing by zero for nodes with zero degree
ratio_betweenness_degree = {
    node: betweenness_centrality[node] / degree if degree > 0 else 0
    for node, degree in node_degree.items()
}

# Combine all metrics into a DataFrame for easy analysis
metrics_df = pd.DataFrame({
    'Node': list(disease_lcc.nodes),
    'Node Degree': [node_degree[node] for node in disease_lcc.nodes],
    'Betweenness Centrality': [betweenness_centrality[node] for node in disease_lcc.nodes],
    'Eigenvector Centrality': [eigenvector_centrality[node] for node in disease_lcc.nodes],
    'Closeness Centrality': [closeness_centrality[node] for node in disease_lcc.nodes],
    'Betweenness/Degree': [ratio_betweenness_degree[node] for node in disease_lcc.nodes]
})

print(metrics_df.describe())

       Node Degree  Betweenness Centrality  Eigenvector Centrality  \
count   47.000000          47.000000               47.000000         
mean     4.808511           0.038175                0.112414         
std      4.100044           0.067539                0.093954         
min      1.000000           0.000000                0.002788         
25%      2.000000           0.000000                0.043515         
50%      4.000000           0.009700                0.097069         
75%      6.000000           0.043816                0.149719         
max     22.000000           0.350636                0.465064         

       Closeness Centrality  Betweenness/Degree  
count        47.000000            47.000000      
mean          0.381920             0.005007      
std           0.072194             0.006116      
min           0.237113             0.000000      
25%           0.343303             0.000000      
50%           0.380165             0.002260      
75%           0.433

In [89]:
# Sort the DataFrame by Node Degree in descending order
metrics_df_sorted = metrics_df.sort_values(by='Node Degree', ascending=False)

# Select the first 47 nodes
top_47_metrics = metrics_df_sorted.head(47)

# Save the top 47 metrics to a CSV file
top_47_metrics.to_csv('top_47_disease_lcc_metrics.csv', index=False)

# Display the table
print(top_47_metrics)

      Node     Node Degree  Betweenness Centrality  Eigenvector Centrality  \
8        ESR2      22              0.350636                0.465064          
4        ESR1      13              0.128508                0.304440          
6         SRC      13              0.181352                0.314973          
24       ENO1      12              0.224414                0.256062          
12      GAPDH      11              0.100957                0.237012          
34      PARK7      10              0.065251                0.137031          
36       TPI1       8              0.086163                0.153290          
13       CCT2       7              0.032903                0.191723          
19      ACTG1       7              0.042035                0.169749          
2        CAP1       6              0.107311                0.141101          
25        VCL       6              0.025448                0.194869          
7        P4HB       6              0.079007                0.157

In [93]:
# Save the table to a CSV file
output_path = r'E:\University\BINM\top_47_disease_lcc_metrics.csv'
top_47_metrics.to_csv(output_path, index=False)

print("Table saved as 'top_47_disease_lcc_metrics.csv'")

Table saved as 'top_47_disease_lcc_metrics.csv'


### 2. Comparative analysis of the disease genes identification algorithms

In [175]:
disease_genes_1c = pd.DataFrame(disease_lcc, columns=['Gene'])
# Save the DataFrame to a TSV file 
outputpath = r'E:\University\BINM\84_disease_genes.tsv'
disease_genes_1c.to_csv(outputpath, sep='\t', index=False) 
print("Disease genes saved to '84_disease_genes.tsv'")

Disease genes saved to '84_disease_genes.tsv'


In [124]:
disease_edge_list = pd.DataFrame(interactome_lcc.edges)
outputpath = r'E:\University\BINM\interactome_lcc_edgelist.tsv'
disease_edge_list.to_csv(outputpath, sep='\t', index=False) 
print("Disease genes saved to 'interactome_lcc_edgelist.tsv'")

Disease genes saved to 'interactome_lcc_edgelist.tsv'


In [116]:
cd E:\University\BINM

E:\University\BINM


C:\Users\Syed Habibul Bashar\AppData\Roaming\Python\Python39\site-packages\IPython\core\magics\osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [224]:
def load_graph_and_genes(edgelist_file, seed_genes_file):
    """
    Load the graph and seed genes.

    Args:
        edgelist_file (str): Path to the edgelist file.
        seed_genes_file (str): Path to the file containing seed genes.

    Returns:
        tuple: Graph object and list of valid seed genes.
    """
    # Load the graph
    G = nx.read_edgelist(edgelist_file, delimiter="\t")

    # Read the seed genes TSV file
    seed_genes_df = pd.read_csv(seed_genes_file, delimiter="\t")
    seed_genes = seed_genes_df['Disease Genes in LCC'].tolist()

    # Validate seed genes
    valid_seed_genes = [gene for gene in seed_genes if gene in G.nodes()]
    return G, valid_seed_genes


In [197]:
def diffusion_ranking_sparse_exclude_seeds(G, seed_genes, t):
    """
    Perform diffusion-based ranking using sparse matrices and exclude seed genes.

    Args:
        G (networkx.Graph): Input graph.
        seed_genes (list): List of seed genes.
        t (float): Diffusion time.

    Returns:
        list: Ranked list of nodes based on diffusion scores, excluding seed genes.
    """
    nodes = list(G.nodes())
    node_to_idx = {node: idx for idx, node in enumerate(nodes)}
    A = nx.adjacency_matrix(G, nodelist=nodes)
    L = csgraph.laplacian(A, normed=False)

    H0 = np.zeros(len(nodes))
    for gene in seed_genes:
        if gene in node_to_idx:
            H0[node_to_idx[gene]] = 1

    Ht = expm_multiply(-t * L, H0)
    ranked_nodes = [nodes[idx] for idx in np.argsort(-Ht)]
    ranked_nodes = [node for node in ranked_nodes if node not in seed_genes]
    return ranked_nodes

In [198]:
def diamond_ranking(G, seed_genes, max_iterations=50):
    """
    Perform DIAMOnD algorithm to expand the seed genes.

    Args:
        G (networkx.Graph): Input graph.
        seed_genes (list): List of seed genes.
        max_iterations (int): Number of nodes to add.

    Returns:
        list: Ranked list of additional nodes based on significance.
    """
    added_nodes = []
    seed_set = set(seed_genes)

    for _ in range(max_iterations):
        candidate_nodes = [node for node in G.nodes if node not in seed_set]
        node_pvalues = {}

        for node in candidate_nodes:
            neighbors = set(G.neighbors(node))
            overlap = len(neighbors & seed_set)
            M = len(G.nodes)
            n = len(seed_set)
            N = len(neighbors)
            x = overlap
            p_value = hypergeom.sf(x - 1, M, n, N)
            node_pvalues[node] = p_value

        if not node_pvalues:
            break
        next_node = min(node_pvalues, key=node_pvalues.get)
        seed_set.add(next_node)
        added_nodes.append(next_node)

    return added_nodes

In [200]:
def diable_ranking(G, seed_genes, max_iterations=50):
    """
    Perform DiaBLE algorithm with adjusted universe expansion.

    Args:
        G (networkx.Graph): Input graph.
        seed_genes (list): List of seed genes.
        max_iterations (int): Number of iterations.

    Returns:
        list: Ranked list of additional nodes based on significance.
    """
    added_nodes = []
    seed_set = set(seed_genes)

    for _ in range(max_iterations):
        candidate_nodes = set()
        for seed in seed_set:
            candidate_nodes.update(G.neighbors(seed))

        candidate_nodes = [node for node in candidate_nodes if node not in seed_set]
        node_pvalues = {}

        for node in candidate_nodes:
            neighbors = set(G.neighbors(node))
            overlap = len(neighbors & seed_set)
            M = len(G.nodes)
            n = len(seed_set)
            N = len(neighbors)
            x = overlap
            p_value = hypergeom.sf(x - 1, M, n, N)
            node_pvalues[node] = p_value

        if not node_pvalues:
            break
        next_node = min(node_pvalues, key=node_pvalues.get)
        seed_set.add(next_node)
        added_nodes.append(next_node)

    return added_nodes

In [201]:
def evaluate_performance(predicted_genes, true_genes, top_k):
    """
    Evaluate precision, recall, and F1-score at top-k predictions.

    Args:
        predicted_genes (list): Ranked list of predicted genes.
        true_genes (set): Set of true disease genes.
        top_k (int): Number of top predictions to evaluate.

    Returns:
        dict: Precision, Recall, and F1-score.
    """
    predicted_set = set(predicted_genes[:top_k])
    true_positives = len(predicted_set & true_genes)
    false_positives = len(predicted_set - true_genes)
    false_negatives = len(true_genes - predicted_set)

    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

    return {"precision": precision, "recall": recall, "f1": f1}

In [202]:
def cross_validate_ranking(G, seed_genes, true_genes, algorithm, params, top_k, folds=5):
    """
    Perform cross-validation for a ranking algorithm.

    Args:
        G (networkx.Graph): Input graph.
        seed_genes (list): List of seed genes.
        true_genes (set): Set of true disease genes.
        algorithm (callable): Ranking algorithm to evaluate.
        params (dict): Parameters for the ranking algorithm.
        top_k (int): Number of top predictions to evaluate.
        folds (int): Number of cross-validation folds.

    Returns:
        dict: Average and standard deviation of precision, recall, and F1-score.
    """
    seed_genes = list(seed_genes)
    np.random.shuffle(seed_genes)
    fold_size = len(seed_genes) // folds
    precision_scores, recall_scores, f1_scores = [], [], []

    for i in range(folds):
        test_set = set(seed_genes[i * fold_size:(i + 1) * fold_size])
        train_set = set(seed_genes) - test_set

        predicted_genes = algorithm(G, list(train_set), **params)
        metrics = evaluate_performance(predicted_genes, test_set, top_k)
        precision_scores.append(metrics["precision"])
        recall_scores.append(metrics["recall"])
        f1_scores.append(metrics["f1"])

    return {
        "precision": (np.mean(precision_scores), np.std(precision_scores)),
        "recall": (np.mean(recall_scores), np.std(recall_scores)),
        "f1": (np.mean(f1_scores), np.std(f1_scores)),
    }

In [203]:
def compare_algorithms(G, valid_seed_genes, true_genes, top_k, folds=5):
    algorithms = {
        "DIAMOnD": {
            "func": diamond_ranking,
            "params": {"max_iterations": top_k},
        },
        "DiaBLE": {
            "func": diable_ranking,
            "params": {"max_iterations": top_k},
        },
        "Diffusion_t=0.002": {
            "func": diffusion_ranking_sparse_exclude_seeds,
            "params": {"t": 0.002},
        },
        "Diffusion_t=0.005": {
            "func": diffusion_ranking_sparse_exclude_seeds,
            "params": {"t": 0.005},
        },
        "Diffusion_t=0.01": {
            "func": diffusion_ranking_sparse_exclude_seeds,
            "params": {"t": 0.01},
        },
    }

    results = {}
    for name, algorithm in algorithms.items():
        print(f"Running cross-validation for {name}...")
        results[name] = cross_validate_ranking(
            G,
            valid_seed_genes,
            true_genes,
            algorithm["func"],
            algorithm["params"],
            top_k,
            folds,
        )

    return results

In [225]:
if __name__ == "__main__":
    # Load the graph and seed genes
    edgelist_file = "interactome_lcc_edgelist.tsv"
    seed_genes_file = "84_disease_genes.tsv"
    G, valid_seed_genes = load_graph_and_genes(edgelist_file, seed_genes_file)
    true_genes = set(valid_seed_genes)

    # Evaluation parameters
    top_k = 50  # Evaluate top 50 predictions
    folds = 5

    # Compare all algorithms
    print("Comparing DIAMOnD, DiaBLE, and Diffusion-based algorithms...")
    results = compare_algorithms(G, valid_seed_genes, true_genes, top_k, folds)

    # Output results
    for algorithm, metrics in results.items():
        print(f"\nAlgorithm: {algorithm}")
        print(f"Precision: {metrics['precision'][0]:.4f} ± {metrics['precision'][1]:.4f}")
        print(f"Recall: {metrics['recall'][0]:.4f} ± {metrics['recall'][1]:.4f}")
        print(f"F1-score: {metrics['f1'][0]:.4f} ± {metrics['f1'][1]:.4f}")

Comparing DIAMOnD, DiaBLE, and Diffusion-based algorithms...
Running cross-validation for DIAMOnD...
Running cross-validation for DiaBLE...
Running cross-validation for Diffusion_t=0.002...
Running cross-validation for Diffusion_t=0.005...
Running cross-validation for Diffusion_t=0.01...

Algorithm: DIAMOnD
Precision: 0.0200 ± 0.0126
Recall: 0.0625 ± 0.0395
F1-score: 0.0303 ± 0.0192

Algorithm: DiaBLE
Precision: 0.0160 ± 0.0196
Recall: 0.0500 ± 0.0612
F1-score: 0.0242 ± 0.0297

Algorithm: Diffusion_t=0.002
Precision: 0.0080 ± 0.0098
Recall: 0.0250 ± 0.0306
F1-score: 0.0121 ± 0.0148

Algorithm: Diffusion_t=0.005
Precision: 0.0080 ± 0.0098
Recall: 0.0250 ± 0.0306
F1-score: 0.0121 ± 0.0148

Algorithm: Diffusion_t=0.01
Precision: 0.0000 ± 0.0000
Recall: 0.0000 ± 0.0000
F1-score: 0.0000 ± 0.0000


### 3.Putative disease gene identification

In [227]:
if __name__ == "__main__":
    # Load the graph and seed genes
    edgelist_file = "interactome_lcc_edgelist.tsv"
    seed_genes_file = "84_disease_genes.tsv"
    G, valid_seed_genes = load_graph_and_genes(edgelist_file, seed_genes_file)

    # DIAMOnD parameters
    max_iterations = 100  # We need the top 100 putative disease genes

    print("Running DIAMOnD algorithm...")

    # Run the DIAMOnD algorithm with all known GDAs as seed genes
    predicted_genes = diamond_ranking(G, valid_seed_genes, max_iterations=max_iterations)

    # Exclude seed genes from the results
    putative_genes = [gene for gene in predicted_genes if gene not in valid_seed_genes]

    # Get the top 100 putative disease genes
    top_100_putative_genes = putative_genes[:100]

    # Save the results
    output_file = "top_100_putative_disease_genes_diamond.csv"
    pd.DataFrame(top_100_putative_genes, columns=["Gene"]).to_csv(output_file, index=False)

    print(f"Top 100 putative disease genes saved to '{output_file}'")

Running DIAMOnD algorithm...
Top 100 putative disease genes saved to 'top_100_putative_disease_genes_diamond.csv'


Using the provided script in github!

In [228]:
!python DiaMond.py interactome_lcc_edgelist.tsv 84_disease_genes.tsv 100 1 Diamond_result.tsv


E:\University\BINM\DiaMond.py:368: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  p = float(DIAMOnD_node_info[3])


DIAMOnD(): ignoring 1 of 85 seed genes that are not in the network

 results have been saved to 'Diamond_result.tsv' 



3.2 EnrichR

In [231]:
cd E:\University\BINM\EnrichR

E:\University\BINM\EnrichR


C:\Users\Syed Habibul Bashar\AppData\Roaming\Python\Python39\site-packages\IPython\core\magics\osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [240]:
def load_and_filter_enrichment(file, p_value_threshold=0.05):
    """
    Load and filter EnrichR results by adjusted p-value.

    Args:
        file (str): Path to the EnrichR results file.
        p_value_threshold (float): Threshold for filtering adjusted p-values.

    Returns:
        DataFrame: Filtered DataFrame with significant terms.
    """
    # Load the EnrichR results (tab-delimited)
    df = pd.read_csv(file, delimiter="\t")

    # Check if required columns exist
    if "Adjusted P-value" not in df.columns or "Term" not in df.columns:
        raise ValueError(f"File {file} must contain 'Adjusted P-value' and 'Term' columns.")

    # Filter by adjusted p-value
    filtered_df = df[df["Adjusted P-value"] < p_value_threshold]

    return filtered_df[["Term", "Adjusted P-value"]]

def find_overlap(filtered_df1, filtered_df2):
    """
    Find overlap between two filtered enrichment result DataFrames.

    Args:
        filtered_df1 (DataFrame): Filtered results for the first dataset.
        filtered_df2 (DataFrame): Filtered results for the second dataset.

    Returns:
        DataFrame: DataFrame containing the overlapping terms.
    """
    # Merge the two DataFrames on the "Term" column
    overlap = pd.merge(
        filtered_df1, filtered_df2, on="Term", suffixes=("_original", "_putative")
    )
    return overlap

def process_and_evaluate_all(original_files, putative_files, categories, output_file):
    """
    Process all enrichment files, perform overlap evaluation, and save combined results to a single CSV.

    Args:
        original_files (list): List of file paths for original disease genes.
        putative_files (list): List of file paths for putative disease genes.
        categories (list): List of categories (e.g., GO-BP, GO-MF).
        output_file (str): Path to save the combined analysis results as a CSV file.

    Returns:
        None
    """
    # List to store all results
    combined_results = []

    for original_file, putative_file, category in zip(original_files, putative_files, categories):
        print(f"Processing category: {category}")

        # Load and filter the enrichment results
        filtered_original = load_and_filter_enrichment(original_file)
        filtered_putative = load_and_filter_enrichment(putative_file)

        # Find overlapping terms
        overlap = find_overlap(filtered_original, filtered_putative)

        # Extract sets of terms
        original_terms = set(filtered_original["Term"])
        putative_terms = set(filtered_putative["Term"])
        overlap_terms = set(overlap["Term"])

        # Calculate metrics
        num_original_terms = len(original_terms)
        num_putative_terms = len(putative_terms)
        num_overlap_terms = len(overlap_terms)
        percentage_overlap_original = (num_overlap_terms / num_original_terms) * 100 if num_original_terms > 0 else 0
        percentage_overlap_putative = (num_overlap_terms / num_putative_terms) * 100 if num_putative_terms > 0 else 0
        unique_to_original = original_terms - overlap_terms
        unique_to_putative = putative_terms - overlap_terms

        # Append results to the combined list
        combined_results.append({
            "Category": category,
            "Total Original Terms": num_original_terms,
            "Total Putative Terms": num_putative_terms,
            "Number of Overlapping Terms": num_overlap_terms,
            "Percentage Overlap (Original)": percentage_overlap_original,
            "Percentage Overlap (Putative)": percentage_overlap_putative,
            "Unique Terms in Original": len(unique_to_original),
            "Unique Terms in Putative": len(unique_to_putative),
        })

    # Convert combined results to a DataFrame and save to CSV
    combined_results_df = pd.DataFrame(combined_results)
    combined_results_df.to_csv(output_file, index=False)
    print(f"Combined analysis results saved to {output_file}")

# Example Usage
if __name__ == "__main__":
    # Define file paths for original and putative results
    original_files = [
        "GO_BP_original.txt",
        "GO_MF_original.txt",
        "GO_CC_original.txt",
        "Reactome_original.txt",
        "KEGG_original.txt",
    ]
    putative_files = [
        "GO_BP_putative.txt",
        "GO_MF_putative.txt",
        "GO_CC_putative.txt",
        "Reactome_putative.txt",
        "KEGG_putative.txt",
    ]

    # Define categories
    categories = ["GO-BP", "GO-MF", "GO-CC", "Reactome", "KEGG"]

    # Output file for combined results
    output_file = "combined_analysis_results.csv"

    # Process all files and evaluate overlaps
    process_and_evaluate_all(original_files, putative_files, categories, output_file)


Processing category: GO-BP
Processing category: GO-MF
Processing category: GO-CC
Processing category: Reactome
Processing category: KEGG
Combined analysis results saved to combined_analysis_results.csv


### 4. Drugs repurposing

In [ ]:
cd E:\University\BINM\

E:\University\BINM


C:\Users\Syed Habibul Bashar\AppData\Roaming\Python\Python39\site-packages\IPython\core\magics\osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [257]:
# Load the TSV files
interactions_file = "Interactions.tsv"
first_20_genes_file = "First_20_Gene.txt"

interactions = pd.read_csv(interactions_file, delimiter="\t")
first_20_genes = pd.read_csv(first_20_genes_file, delimiter="\t")

# Ensure columns exist
if "gene_name" not in interactions.columns or "drug_name" not in interactions.columns or "approved" not in interactions.columns:
    raise ValueError("Interactions file must contain 'gene_name', 'drug_name', and 'approved' columns.")
if "Gene" not in first_20_genes.columns:
    raise ValueError("First_20_Gene file must contain a 'Gene' column.")

# Find genes in "First_20_Gene" that are in "Interactions"
first_20_gene_list = set(first_20_genes["Gene"])
filtered_interactions = interactions[interactions["gene_name"].isin(first_20_gene_list)]

# Filter for approved drugs (approved=True)
approved_drugs = filtered_interactions[filtered_interactions["approved"] == True]

# Select relevant columns (gene_name, drug_name, approved)
result = approved_drugs[["gene_name", "drug_name", "approved"]].drop_duplicates()

# Save the results to a new CSV file
output_file = "Gene_to_Approved_Drugs_With_Status.csv"
result.to_csv(output_file, index=False)

# Print the results
print(f"Number of gene-drug associations found: {len(result)}")
print(f"Results saved to '{output_file}'")
print(result.head())

Number of gene-drug associations found: 35
Results saved to 'Gene_to_Approved_Drugs_With_Status.csv'
      gene_name   drug_name   approved
773      NME2      LAMIVUDINE   True  
2973     NME2       TENOFOVIR   True  
11449    CDK2      LOVASTATIN   True  
13884     FN1     OCRIPLASMIN   True  
14078    CFL1    CLOTRIMAZOLE   True  


In [260]:
# Remove duplicate (gene_name, drug_name) pairs to ensure unique associations
approved_drugs_unique = approved_drugs.drop_duplicates(subset=["gene_name", "drug_name"])

# Group by gene_name and aggregate associated drugs
gene_drug_summary = approved_drugs_unique.groupby("gene_name")["drug_name"].apply(list).reset_index(name="associated_drugs")

# Add a count column for the number of drugs associated with each gene
gene_drug_summary["drug_count"] = gene_drug_summary["associated_drugs"].apply(len)

# Save the summary to a CSV file
summary_file = "Gene_Drug_Summary.csv"
gene_drug_summary.to_csv(summary_file, index=False)

# Print the summary
print(f"Gene-drug summary saved to '{summary_file}'")
print(gene_drug_summary)


Gene-drug summary saved to 'Gene_Drug_Summary.csv'
  gene_name                  associated_drugs                   drug_count
0    BAP1    [PANOBINOSTAT, VORINOSTAT, SUNITINIB, EVEROLIM...       6    
1    CDK2    [LOVASTATIN, PACLITAXEL, RESVERATROL, DAUNORUB...       9    
2    CFL1    [CLOTRIMAZOLE, CINNARIZINE, CLOFIBRATE, FENOFI...       5    
3    EZH2    [TAZEMETOSTAT, DABRAFENIB, TAZEMETOSTAT HYDROB...       3    
4     FN1                           [OCRIPLASMIN, DACARBAZINE]       2    
5    NME2    [LAMIVUDINE, TENOFOVIR, ZIDOVUDINE, PROGESTERO...       5    
6    SOD1    [TETRACYCLINE, DOXYCYCLINE ANHYDROUS, TOFERSEN...       4    
7     VCP                                    [HEXACHLOROPHENE]       1    


### 5. ProConSuL vs DIAMOnD (Best performer)

In [3]:
!python proconsul.py --network_file interactome_lcc_edgelist.tsv --seed_file 84_disease_genes.tsv --n 20 --outfile_name ProConSuL_results.csv --temp 1

PROCONSUL(): ignoring 1 of 48 seed genes that are not in the network
PROCONSUL(): Round 1/10
PROCONSUL(): Round 2/10
PROCONSUL(): Round 3/10
PROCONSUL(): Round 4/10
PROCONSUL(): Round 5/10
PROCONSUL(): Round 6/10
PROCONSUL(): Round 7/10
PROCONSUL(): Round 8/10
PROCONSUL(): Round 9/10
PROCONSUL(): Round 10/10

 results have been saved to 'ProConSuL_results.csv' 



processing the outputfile for comarision with the DIAMOnD nodes

In [7]:
# Load the data (adjust the file name and delimiter as needed)
file_path = "ProConSuL_results.csv"  # Replace with your file name
data = pd.read_csv(file_path, delimiter="\t")

# Extract the 'node' column
node_column = data["node"]

# Save the node column to a new file
output_file = "ProConSuL_results_Nodes.csv"
node_column.to_csv(output_file, index=False)

# Print success message
print(f"Node column saved to {output_file}")

Node column saved to ProConSuL_results_Nodes.csv


In [8]:
# Load ProConSuL results
proconsul_results_file = "ProConSuL_results_Nodes.csv"  # Replace with your actual file
proconsul_results = pd.read_csv(proconsul_results_file)
top_20_proconsul_genes = proconsul_results.head(20)['node'].tolist()

# Load DIAMOnD results
diamond_results_file = "100_putative_gene.tsv"  # Use the DIAMOnD top 20 file
diamond_results = pd.read_csv(diamond_results_file)
top_20_diamond_genes = diamond_results.head(20)['DIAMOnD_node'].tolist()

# Compare the two lists
overlap = set(top_20_proconsul_genes).intersection(set(top_20_diamond_genes))
unique_to_proconsul = set(top_20_proconsul_genes) - set(top_20_diamond_genes)
unique_to_diamond = set(top_20_diamond_genes) - set(top_20_proconsul_genes)

# Print results
print("Overlap:", overlap)
print("Unique to ProConSuL:", unique_to_proconsul)
print("Unique to DIAMOnD:", unique_to_diamond)


Overlap: {'ACO2', 'VCP', 'SOD1', 'CFL1', 'CDK2', 'EZH2', 'CLIC1', 'CLIC4', 'BAP1', 'DSTN', 'UBE2M', 'PRDX6', 'AGR2', 'PRDX2'}
Unique to ProConSuL: {'U2AF2', 'HSPA8', 'MCM2', 'ISG15', 'RICTOR', 'IQGAP1'}
Unique to DIAMOnD: {'NME2', 'FN1', 'LDHA', 'TKT', 'MYOC', 'TIMP2'}
